# Web Scraping Using Beautiful Soup
---

Mount gdrive folder

In [75]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Change directory to the working directory

In [76]:
%cd /content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql

/content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql


Import Libraries

In [122]:
# import beautiful soup
from bs4 import BeautifulSoup

import requests
# import requests

# import time
import time

Define `fetch_html` function that fetch HTML text from given URL per pages

In [123]:
def fetch_html(page):
  base_url = 'https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=python'
  page_param = f'&sequence={page}'
  html_content = requests.get(f'{base_url}{page_param}')
  html_text = html_content.text
  
  return html_text

Define `parse_html` function that parse HTML text using **beautiful soup** & **lxml** parser

In [124]:
def parse_html(html_text):
  soup = BeautifulSoup(html_text, 'lxml')

  return soup

Get user input

In [125]:
print('Input how many pages you want to scrape?')
page_number = input('> ')

Input how many pages you want to scrape?
> 3


Define `find_jobs()` function that scraping **company**, **skill**, & **publish date** info

In [126]:
def find_jobs(soup):
  jobs = soup.find_all('li', class_ = 'clearfix job-bx wht-shd-bx')

  list_of_dict_jobs = []

  for idx, job in enumerate(jobs):
    company = job.find('h3', class_ = 'joblist-comp-name').text.replace(' ', ' ').strip()
    skills = job.find('span', class_ = 'srp-skills').text.replace(' ', '').strip()
    publish_date = job.find('span', class_ = 'sim-posted').text
    more_info = job.header.h2.a['href']

    dict_job = {}
    dict_job['company'] = company
    dict_job['skills'] = skills
    dict_job['publish_date'] = publish_date
    dict_job['more_info'] = more_info

    list_of_dict_jobs.append(dict_job)

  return list_of_dict_jobs

Define `write_to_file` function that writes jobs into `.txt` file for every page

In [127]:
def write_to_file(jobs, page):
  with open(f"posts/{page}.txt", "a") as f:
    for job in jobs:
      f.write(f"company name: {job.get('company')} \n")
      f.write(f"skills: {job.get('skills')} \n")
      f.write(f"more info: {job.get('more_info')} \n")
      f.write(f"--------------------------------------- \n")
    
  print(f"file saved: {page}")

Main Code

In [128]:
for idx in range(int(page_number)):
  html_text = fetch_html(idx+1)
  soup = parse_html(html_text)
  jobs = find_jobs(soup)
  write_to_file(jobs, idx+1)

file saved: 1
file saved: 2
file saved: 3
